In [1]:
! pip install transformers accelerate einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 45.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 771.9/771.9 kB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 64.0 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 kB 29.9 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip


In [35]:
import torch
from transformers import InstructBlipForConditionalGeneration, InstructBlipProcessor # install from git repo

model = InstructBlipForConditionalGeneration.from_pretrained(
	"Salesforce/instructblip-vicuna-7b",
	load_in_4bit=False,
	torch_dtype=torch.bfloat16
	)

processor = InstructBlipProcessor.from_pretrained(
	"Salesforce/instructblip-vicuna-7b")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [42]:
import torch

del pipeline

torch.cuda.empty_cache()

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("yay")
else:
    device = torch.device("cpu")

model = model.to(device)

yay


In [43]:
from PIL import Image

image = Image.open("vip1.png")

prompt1 = "describe this image in full detail. describe each and every aspect of the image so that an artist can recreate this image."
prompt2 = "create an extensive description of this image"

answers = []
for _prompt in [prompt1, prompt2]:
    inputs = processor(images=image, 
                        text=_prompt, 
                        return_tensors="pt"
                    ).to(model.device, torch.bfloat16)
    outputs = model.generate(
            **inputs, 
            do_sample=False,
            num_beams=10,
            max_length=512,
            min_length=16,
            top_p=0.9,
            repetition_penalty=1.5,
            temperature=1
        )
    generated_text = processor.batch_decode(
            outputs,
            skip_special_tokens=True
        )[0].strip()
    answers.append(generated_text)
    print(f"{_prompt}: {generated_text}")

describe this image in full detail. describe each and every aspect of the image so that an artist can recreate this image.: In the image, there is a man standing in the middle of a city street. He is wearing a blue shirt and has a serious expression on his face. The sun is setting in the background, casting a warm glow over the scene. There are several cars parked along the street, adding to the urban atmosphere.  In addition to the man and the cars, there are two other people visible in the scene. One person is located on the left side of the image, while the other person is situated on the right side. Both individuals appear to be engaged in their own activities, contributing to the lively atmosphere of the street.
create an extensive description of this image: a man standing in the middle of a street with a sunset in the background


In [2]:
answers = ['In the image, there is a man standing in the middle of a city street. He is wearing a blue shirt and has a serious expression on his face. The sun is setting in the background, casting a warm glow over the scene. There are several cars parked along the street, adding to the urban atmosphere.  In addition to the man and the cars, there are two other people visible in the scene. One person is located on the left side of the image, while the other person is situated on the right side. Both individuals appear to be engaged in their own activities, contributing to the lively atmosphere of the street.',
 'a man standing in the middle of a street with a sunset in the background']

In [47]:
! rm -rf .cache/huggingface/hub/

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [3]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("yay")
else:
    device = torch.device("cpu")

yay


In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

model_name = "tiiuae/falcon-7b-instruct" # "tiiuae/falcon-40b-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)

# del model

torch.cuda.empty_cache()

pipeline = transformers.pipeline(
	"text-generation",
	model=model_name,
	tokenizer=tokenizer,
	torch_dtype=torch.bfloat16,
	trust_remote_code=True,
	device_map=device
)

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- configuration_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- modelling_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [23]:
prompt3 = "give an instagram caption for this photo."
prompt4 = "give 3 emojis relevant to this photo"
prompt5 = "give an instagram hashtags for this photo"
prompt6 = "use chain of reasoning to first derive the instagram caption for this photo, then find the relevant hashtags to put, \
and then re-edit the whole caption to put relevant emojis in it"

from pprint import pprint

for answer in answers[1:]:
	print(answer)
	print()
	print("-"*100)
	for prompt in [prompt3]:
		sequences = pipeline(
			"You are the best Instagram caption writer in the world. For an image with description: " + answer + " " + prompt,
			max_length=200,
			do_sample=True,
			top_k=1,
			num_return_sequences=1,
			eos_token_id=tokenizer.eos_token_id
		)
		pprint(sequences[0]['generated_text'].split('\n'))
		print()
		print("-"*100)
		print()

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


a man standing in the middle of a street with a sunset in the background

----------------------------------------------------------------------------------------------------
['You are the best Instagram caption writer in the world. For an image with '
 'description: a man standing in the middle of a street with a sunset in the '
 'background give an instagram caption for this photo.',
 'The sun is setting, painting the sky with hues of gold and orange, casting a '
 'warm glow on the street below. #sunset #goldenhour #orange']

----------------------------------------------------------------------------------------------------

